In [3]:
# Importing libraries
import pandas as pd
import tensorflow as tf
import numpy as np
import keras

In [4]:
consumption = pd.read_csv("consumption.csv")

In [22]:
features = ['Watt']
data = consumption[features].values.astype(np.float32)
data -= np.mean(data, axis=0)
data /= np.std(data, axis=0)
sequence_length = 2160

def data_generator(batch_size):
    while True:
        idx = sequence_length * np.random.randint(1,len(data)//sequence_length)
        sequence = data[idx-sequence_length:(idx+1)*batch_size]
        X = sequence[:-1]
        y = sequence[1:, :5]
        X = np.expand_dims(X, axis=0)
        y = np.expand_dims(y, axis=0)
        yield X, y

# Create the model
hidden_size = 128
num_classes = 5
num_features = 1
model = keras.Sequential()
model.add(keras.layers.LSTM(hidden_size, input_shape=(sequence_length, 1), return_sequences=True))
model.add(keras.layers.Dense(num_classes, activation='softmax'))
model.compile(loss='categorical_crossentropy', optimizer='adam')
batch_size = 32
num_epochs = 10
steps_per_epoch = 1000
data_gen = data_generator(batch_size)
model.fit(data_gen, steps_per_epoch=steps_per_epoch, epochs=num_epochs)

Epoch 1/50


In [17]:
# We make the predictions
test_consumption = pd.read_csv("test.csv")
data_test = test_consumption['Watt'].values.astype(np.float32)
data_test -= np.mean(data_test, axis=0)
data_test /= np.std(data_test, axis=0)
sequence_length = 2160

def data_split():
    for i in range(1,len((data_test)//sequence_length)+1):
        idx = i*sequence_length
        sequence = data_test[idx-sequence_length:idx]
        X = sequence
        X = np.expand_dims(X, axis=0)
        yield X
df_parts = data_split()
sequence_length = 2160

y_pred_list = []
curves_num = len(test_consumption)//sequence_length
i = 1
for X in data_split() :
    if i > len(test_consumption)//sequence_length:
        break
    y_pred_part = model.predict(np.array(X).reshape(1, -1, 1))
    y_pred_list.append(y_pred_part)
    i = i + 1
y_pred = np.concatenate(y_pred_list, axis=0)

iteration 1/2488
1/1 [==============================] - 0s 73ms/step
iteration 2/2488
1/1 [==============================] - 0s 71ms/step
iteration 3/2488
1/1 [==============================] - 0s 71ms/step
iteration 4/2488
1/1 [==============================] - 0s 84ms/step
iteration 5/2488
1/1 [==============================] - 0s 75ms/step
iteration 6/2488
1/1 [==============================] - 0s 73ms/step
iteration 7/2488
1/1 [==============================] - 0s 71ms/step
iteration 8/2488
1/1 [==============================] - 0s 87ms/step
iteration 9/2488
1/1 [==============================] - 0s 82ms/step
iteration 10/2488
1/1 [==============================] - 0s 82ms/step
iteration 11/2488
1/1 [==============================] - 0s 85ms/step
iteration 12/2488
1/1 [==============================] - 0s 81ms/step
iteration 13/2488
1/1 [==============================] - 0s 82ms/step
iteration 14/2488
1/1 [==============================] - 0s 74ms/step
iteration 15/2488
1/1 [======

In [18]:
print(len(y_pred))

2488


In [3]:
# The predictions are saved in a CSV file
df_pred = pd.DataFrame(pd.DataFrame(data=np.array(y_pred).reshape(-1, 5), columns=['Washing Machine', 'Dishwasher', 'Tumble Dryer', 'Microwave', 'Kettle']))
df_pred = df_pred.round(2)
display(df_pred)
df_pred.to_csv('predictions.csv', index=True,index_label="Index")

,Washing Machine,Dishwasher,Tumble Dryer,Microwave,Kettle
0,0.04,0.06,0.01,0.01,0.01
1,0.02,0.01,0.00,0.00,0.00
2,0.03,0.01,0.00,0.00,0.00
3,0.05,0.02,0.00,0.00,0.00
4,0.05,0.02,0.01,0.00,0.00
...,...,...,...,...,...
5374075,0.01,0.90,0.00,0.00,0.00
5374076,0.01,0.90,0.00,0.00,0.00
5374077,0.01,0.90,0.00,0.00,0.00
5374078,0.01,0.90,0.00,0.00,0.00
